<a href="https://colab.research.google.com/github/yvpaulo/PythonMLCovid19_Previsao_Evolucao/blob/main/ModeloCovid19DIOLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Covid19
##Digital Innovation One

In [1]:
import pandas as pd
import numpy as np # lib para computação científica
from datetime import datetime #para trabalhar com datas
import plotly.express as px #para visualização (gráficos)
import plotly.graph_objects as go #para gráficos mais complexos